In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

In [32]:
filenameList = ["callIn14_4", "callOut14_4", "textIn14_4", "textOut14_4", "callIn14_6", "callOut14_6", "textIn14_6", "textOut14_6", "callIn14_12", "callOut14_12", "textIn14_12", "textOut14_12", "callIn14_24", "callOut14_24", "textIn14_24", "textOut14_24"]#, "callAll14_4", "callAll14_6","callAll14_12", "callAll14_24", "textAll14_4", "textAll14_6", "textAll14_12", "textAll14_24"]
filename2List = ["_average_length", "_counts", "_unique_contacts"] #, "_combined"]
split = 1 #needs to be 2 for 0-7 labels but 1 for label 8
label = "8" #this is the question!!! From 0 to 8

for filename in filenameList:
    print(filename)

    data = pd.read_csv("data/" + filename + filename2 + ".csv")
    data = data[:-10]

    print(data.shape)
    data = data.replace(np.inf, np.nan).dropna(axis=1)
    print(data.shape)

callIn14_4
(172, 187)
(172, 186)
callOut14_4
(187, 187)
(187, 186)
textIn14_4
(280, 187)
(280, 186)
textOut14_4
(89, 187)
(89, 186)
callIn14_6
(172, 173)
(172, 172)
callOut14_6
(187, 173)
(187, 172)
textIn14_6
(280, 173)
(280, 172)
textOut14_6
(89, 173)
(89, 172)
callIn14_12
(172, 159)
(172, 158)
callOut14_12
(187, 159)
(187, 158)
textIn14_12
(280, 159)
(280, 158)
textOut14_12
(89, 159)
(89, 158)
callIn14_24
(172, 152)
(172, 151)
callOut14_24
(187, 152)
(187, 151)
textIn14_24
(280, 152)
(280, 151)
textOut14_24
(89, 152)
(89, 151)


In [24]:
featureE = "PCA"
modelTypelist = ["NB","LR","SVC", "RF"]
dfindex = -10

filenameList = ["callIn14_4", "callOut14_4", "textIn14_4", "textOut14_4", "callIn14_6", "callOut14_6", "textIn14_6", "textOut14_6", "callIn14_12", "callOut14_12", "textIn14_12", "textOut14_12", "callIn14_24", "callOut14_24", "textIn14_24", "textOut14_24"]#, "callAll14_4", "callAll14_6","callAll14_12", "callAll14_24", "textAll14_4", "textAll14_6", "textAll14_12", "textAll14_24"]
filename2List = ["_average_length", "_counts", "_unique_contacts"] #, "_combined"]
split = 1 #needs to be 2 for 0-7 labels but 1 for label 8
label = "8" #this is the question!!! From 0 to 8

for filename in filenameList:
    for filename2 in filename2List:
        
        data = pd.read_csv("data/" + filename + filename2 + ".csv")
        data = data.replace(np.inf, np.nan).dropna(axis=1)
        
        d10 = []
        for i in range(0, data.shape[0]):
            if int(data[label][i]) >= split:
                d10.append(1)
            else:
                d10.append(0)
        data[label] = d10

        #create lists to populate
        flist = [] 
        mlist = []
        llist = []
        featureList = []
        f1List = []
        accuracyList = []
        truePosList = []
        trueNegList = []
        falsePosList = []
        falseNegList = []
        predictions = []
        rseed = []

        for r in range(0,data.shape[0]):
            testdata = data[r:r+1]
            traindata = data.drop([r])

            #limit to features
            testContent = testdata[testdata.columns[:dfindex]]
            print(testContent.shape)
            trainContent = traindata[traindata.columns[:dfindex]]
            print(trainContent.shape)

            #NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION
            min_max_scaler = preprocessing.MinMaxScaler()  
            np_scaled = min_max_scaler.fit_transform(trainContent)
            featureSubset = pd.DataFrame(np_scaled)
            np_scaled2 =  min_max_scaler.transform(testContent)
            testSubset = pd.DataFrame(np_scaled2)

            target = list(traindata[label])
            y_test = list(testdata[label])


            featureDF = []
            testDFs = []

            if featureE == "PCA":
                nFeatureList = list(np.arange(1,6,1))
                for numberOfFeatures in nFeatureList:
                    pca = PCA(n_components=numberOfFeatures)
                    pca = pca.fit(featureSubset)
                    X_pca = pca.transform(featureSubset)
                    pcaDF = pd.DataFrame(X_pca)
                    pcaDF = pcaDF.assign(target = target)
                    featureDF.append(pcaDF)
                    testSubset2 = pca.transform(testSubset)
                    testDFs.append(pd.DataFrame(testSubset2))

            for f in range(0, len(featureDF)):

                train_phq9 = featureDF[f]
                X_test = testDFs[f]

                # upsampling 
                #Count 1s and 0s
                ones = len(train_phq9.loc[train_phq9['target'] == 1])
                zeros = len(train_phq9.loc[train_phq9['target'] == 0])
                if ones >= zeros:
                    majority = 1
                    minority = 0
                else:
                    majority = 0
                    minority = 1


                # Upsample TrainingSet 
                train_majority = train_phq9[train_phq9.target==majority]
                train_minority = train_phq9[train_phq9.target==minority]

                #print("train_majority ="  + str(len(train_majority)))
                #print("train_minority ="  + str(len(train_minority)))

                # Upsample minority class
                train_minority_upsampled = resample(train_minority, 
                                                 replace=True,     # sample with replacement
                                                 n_samples=len(train_majority),    # to match majority class
                                                 random_state=r) # reproducible results

                # Combine majority class with upsampled minority class
                train_phq9 = pd.concat([train_majority, train_minority_upsampled])

                #seperate features and target
                y_train = train_phq9["target"]
                X_train = train_phq9.drop(columns = "target")

                for modelType in modelTypelist:

                    #add data to lists
                    llist.append(label)
                    featureList.append(f +1)
                    mlist.append(modelType)

                    #chose model type
                    if modelType == "SVC":
                        clf = svm.SVC(kernel='rbf', random_state=r)
                    elif modelType == "SVC2":
                        clf = svm.SVC(kernel='linear', random_state=r)
                    elif modelType == "SVC3":
                        clf = svm.SVC(kernel='poly', random_state=r)
                    elif modelType == "SVC4":
                        clf = svm.SVC(kernel='sigmoid', random_state=r)
                    elif modelType == "kNN3":
                        clf = KNeighborsClassifier(n_neighbors=3)
                    elif modelType == "kNN5":
                        clf = KNeighborsClassifier(n_neighbors=5)
                    elif modelType == "kNN1":
                        clf = KNeighborsClassifier(n_neighbors=1)
                    elif modelType == "LR":
                        clf = LogisticRegression(random_state=r)
                    elif modelType == "NB":
                        clf = GaussianNB()
                    elif modelType == "RF":
                        clf = RandomForestClassifier(random_state=r)
                    elif modelType == "XG":
                        clf = xgb.XGBClassifier(random_state=r)
                    else:
                        print("YOU MADE A MISTAKE!!!")


                    #train model and make predictions
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    TP = 0
                    TN = 0
                    FP = 0
                    FN = 0
                    if (y_pred[0]==1) & (y_test[0]==1):
                        TP = 1
                    elif (y_pred[0]==0) & (y_test[0]==0):
                        TN = 1
                    elif (y_pred[0]==1) & (y_test[0]==0):
                        FP = 1
                    elif (y_pred[0]==0) & (y_test[0]==1):
                        FN = 1
                    else:
                        print("ERROR")

                    #populate lists with results
                    truePosList.append(TP)
                    trueNegList.append(TN)
                    falsePosList.append(FP)
                    falseNegList.append(FN)
                    rseed.append(r)

        resultsDF = pd.DataFrame()
        resultsDF["label"] = llist
        resultsDF["model"] = mlist
        resultsDF["nFeatures"] = featureList
        resultsDF["truePos"] = truePosList
        resultsDF["trueNeg"] = trueNegList
        resultsDF["falsePos"] = falsePosList
        resultsDF["falseNeg"] = falseNegList
        resultsDF["testIndex"] = rseed

        resultsDF.to_csv("results/" + filename + filename2 + "Q" + label + "score" + str(split) + ".csv")

(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)


(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)
(1, 176)
(181, 176)


(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)
(1, 176)
(196, 176)


(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)


(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)
(1, 176)
(289, 176)


...

(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(181, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)


(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)
(1, 141)
(196, 141)


(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)


(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)
(1, 141)
(289, 141)


(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98, 141)
(1, 141)
(98